In [ ]:
%matplotlib inline



# Write user defined Operators having third party dependencies
This example shows how advanced DPF python plugins of Operators can be created as standard python packages
and how third party python modules dependencies can be added to the package.

This plugin will hold an Operator which implementation depends on a third party python module named
`gltf <https://pypi.org/project/gltf/>`_. This Operator takes a path, a mesh and 3D vector field in input
and exports the mesh and the norm of the input field in a gltf file located at the given path.


## Write Operator
For this more advanced use case, a python package is created.
Each Operator implementation derives from `ansys.dpf.core.custom_operator.CustomOperatorBase`
and a call to `ansys.dpf.core.custom_operator.record_operator` records the Operators of the plugin.
See the complete package in:



## Load Plugin
Once a python plugin is written as a package, it can be loaded with the function
:py:func:`ansys.dpf.core.core.load_library` taking as first argument the path to the directory of the plugin,
as second argument ``py_`` + any name identifying the plugin,
and as last argument the function's name exposed in the ``__init__.py`` file and used to record operators.



In [4]:
import os
from ansys.dpf import core as dpf
from ansys.dpf.core import examples

tmp = dpf.make_tmp_dir_server()
dpf.upload_files_in_folder(
    dpf.path_utilities.join(tmp, "plugins", "gltf_plugin"),
    dpf.path_utilities.join(os.getcwd(), "..", "plugins", "gltf_plugin")
)
dpf.upload_file(
    dpf.path_utilities.join(os.getcwd(), "..", "plugins", "gltf_plugin.xml"),
    dpf.path_utilities.join(tmp, "plugins", "gltf_plugin.xml")
)

dpf.load_library(
    dpf.path_utilities.join(tmp, "plugins", "gltf_plugin"),
    "py_dpf_gltf",
    "load_operators")

Uploading...: 12057 of 12057 KB 100%|#########################################|


'py_dpf_gltf successfully loaded'

Once the Plugin loaded, Operators recorded in the plugin can be used with:



In [5]:
new_operator = dpf.Operator("gltf_export")

This new Operator ``gltf_export`` requires a triangle surface mesh, a displacement Field on this surface mesh
as well as an export path as inputs.
To demo this new Operator, a `ansys.dpf.core.model.Model` on a simple file is created,
`ansys.dpf.core.operators.mesh.tri_mesh_skin` Operator is used to extract the surface of the mesh in triangles
elements.



## Use the Custom Operator



In [6]:
import tempfile
import os

model = dpf.Model(dpf.upload_file_in_tmp_folder(examples.static_rst))

mesh = model.metadata.meshed_region
skin_mesh = dpf.operators.mesh.tri_mesh_skin(mesh=mesh)

displacement = model.results.displacement()
displacement.inputs.mesh_scoping(skin_mesh)
displacement.inputs.mesh(skin_mesh)
new_operator.inputs.path(os.path.join(tmp, "out"))
new_operator.inputs.mesh(skin_mesh)
new_operator.inputs.field(displacement.outputs.fields_container()[0])
new_operator.run()

print("operator ran successfully")

operator ran successfully


In [7]:
dpf.download_file(os.path.join(tmp, "out.glb"), os.path.join(os.getcwd(), "out.glb"))

Downloading...: 2 of 2 KB 100%|###############################################|
